In [1]:
import io

import os

import sys

import string

import numpy as np

import pandas as pd

from tensorflow import keras

from __future__ import print_function

from tensorflow.keras.models import Sequential

from sklearn.model_selection import train_test_split

from tensorflow.keras.callbacks import LambdaCallback, ModelCheckpoint, EarlyStopping

from tensorflow.keras.layers import Dense, Dropout, Activation, LSTM, Bidirectional, Embedding


translator = str.maketrans('', '', string.punctuation)

df = pd.read_csv("/content/drive/MyDrive/RapLyrics/PopRap-Male/PopRap-Male.csv", sep="\t")

df.head()



,song_id,lyrics
0,5ueUywJbmAZY2MF9hdJS2y,"[""[Verse 1]\nI got a funny feelin'\nThe moment..."
1,49X0LAl6faAusYq02PRAY6,"[""Intéressé(e) par l'explication des paroles d..."
2,3HPx8VWi8spiBTcGeWGP7h,['[Intro]\nThey make me sick\nI know I feel yo...
3,5jDnI5jmAqpvKwmpXP7VSj,"[""[Verse 1]\nI can hear the neighbors\nThey're..."
4,7Hgcw6qYU1ukRdRGvEkJk3,['He promised her a new and better life out in...


In [2]:
pdf = pd.read_csv('/content/drive/MyDrive/RapLyrics/PoetryFoundationData.csv',quotechar='"')

pdf.head()

,Unnamed: 0,Title,Poem,Poet,Tags,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,...,Unnamed: 292,Unnamed: 293,Unnamed: 294,Unnamed: 295,Unnamed: 296,Unnamed: 297,Unnamed: 298,Unnamed: 299,Unnamed: 300,Unnamed: 301,Unnamed: 302,Unnamed: 303,Unnamed: 304,Unnamed: 305,Unnamed: 306,Unnamed: 307,Unnamed: 308,Unnamed: 309,Unnamed: 310,Unnamed: 311,Unnamed: 312,Unnamed: 313,Unnamed: 314,Unnamed: 315,Unnamed: 316,Unnamed: 317,Unnamed: 318,Unnamed: 319,Unnamed: 320,Unnamed: 321,Unnamed: 322,Unnamed: 323,Unnamed: 324,Unnamed: 325,Unnamed: 326,Unnamed: 327,Unnamed: 328,Unnamed: 329,Unnamed: 330,Unnamed: 331
0,0.0,\r\r\n Objects Used to Prop...,"\r\r\nDog bone, stapler,\r\r\ncribbage board, ...",Michelle Menting,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,\r\r\n The New Church\r\r\n...,"\r\r\nThe old cupola glinted above the clouds,...",Lucia Cherciu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.0,\r\r\n Look for Me\r\r\n ...,\r\r\nLook for me under the hood\r\r\nof that ...,Ted Kooser,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3.0,\r\r\n Wild Life\r\r\n ...,"\r\r\nBehind the silo, the Mother Rabbit\r\r\n...",Grace Cavalieri,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4.0,\r\r\n Umbrella\r\r\n ...,\r\r\nWhen I push your button\r\r\nyou fly off...,Connie Wanek,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
def split_text(x):

   text = x['lyrics']
   x = str(x)

   sections = str(text).split('\\n\\n')

   keys = {'Verse 1': np.nan,'Verse 2':np.nan,'Verse 3':np.nan,'Verse 4':np.nan, 'Chorus':np.nan}

   lyrics = str()

   single_text = []

   res = {}

   for s in sections:

       key = s[s.find('[') + 1:s.find(']')].strip()

       if ':' in key:

           key = key[:key.find(':')]
          

       if key in keys:

           single_text += [x.lower().replace('(','').replace(')','').translate(translator) for x in s[s.find(']')+1:].split('\\n') if len(x) > 1]
          

       res['single_text'] =  ' \n '.join(single_text)

   return pd.Series(res)


df = df.join( df.apply(split_text, axis=1))

df.head()

,song_id,lyrics,single_text
0,5ueUywJbmAZY2MF9hdJS2y,"[""[Verse 1]\nI got a funny feelin'\nThe moment...",you shouldnt kiss me like this unless you mean...
1,49X0LAl6faAusYq02PRAY6,"[""Intéressé(e) par l'explication des paroles d...",lady \n hear me tonight \n cause my feeling \n...
2,3HPx8VWi8spiBTcGeWGP7h,['[Intro]\nThey make me sick\nI know I feel yo...,you make me sick \n i want you and i am hating...
3,5jDnI5jmAqpvKwmpXP7VSj,"[""[Verse 1]\nI can hear the neighbors\nThey're...",but for the grace of god go i \n i mustve been...
4,7Hgcw6qYU1ukRdRGvEkJk3,['He promised her a new and better life out in...,


In [4]:
pdf['single_text'] = pdf['Poem'].apply(lambda x: ' \n '.join([l.lower().strip().translate(translator) for l in str(x).splitlines() if len(l)>0]))

pdf.head()

sum_df = pd.DataFrame( df['single_text'] )

sum_df = sum_df.append(pd.DataFrame( pdf['single_text'] ))

sum_df.dropna(inplace=True)

In [5]:
text_as_list = []

frequencies = {}

uncommon_words = set()

MIN_FREQUENCY = 7

MIN_SEQ = 5

BATCH_SIZE =  32


def extract_text(text):

   global text_as_list

   text_as_list += [w for w in text.split(' ') if w.strip() != '' or w == '\n']


sum_df['single_text'].apply( extract_text )

print('Total words: ', len(text_as_list))


for w in text_as_list:

   frequencies[w] = frequencies.get(w, 0) + 1
  

uncommon_words = set([key for key in frequencies.keys() if frequencies[key] < MIN_FREQUENCY])

words = sorted(set([key for key in frequencies.keys() if frequencies[key] >= MIN_FREQUENCY]))


num_words = len(words)

word_indices = dict((w, i) for i, w in enumerate(words))

indices_word = dict((i, w) for i, w in enumerate(words))

print('Words with less than {} appearances: {}'.format( MIN_FREQUENCY, len(uncommon_words)))

print('Words with more than {} appearances: {}'.format( MIN_FREQUENCY, len(words)))


valid_seqs = []

end_seq_words = []

for i in range(len(text_as_list) - MIN_SEQ ):

   end_slice = i + MIN_SEQ + 1

   if len( set(text_as_list[i:end_slice]).intersection(uncommon_words) ) == 0:

       valid_seqs.append(text_as_list[i: i + MIN_SEQ])

       end_seq_words.append(text_as_list[i + MIN_SEQ])
      

print('Valid sequences of size {}: {}'.format(MIN_SEQ, len(valid_seqs)))


X_train, X_test, y_train, y_test = train_test_split(valid_seqs, end_seq_words, test_size=0.02, random_state=42)

print(X_train[2:5])

Total words:  442103
Words with less than 7 appearances: 16436
Words with more than 7 appearances: 3385
Valid sequences of size 5: 318685
[['gotta', 'hold', 'on', 'to', 'your'], ['\n', 'that', 'i', 'rock', 'all'], ['\n', 'i', 'thought', 'you', 'were']]


In [6]:
# Data generator for fit and evaluate

def generator(sentence_list, next_word_list, batch_size):

   index = 0

   while True:

       x = np.zeros((batch_size, MIN_SEQ), dtype=np.int32)

       y = np.zeros((batch_size), dtype=np.int32)

       for i in range(batch_size):

           for t, w in enumerate(sentence_list[index % len(sentence_list)]):

               x[i, t] = word_indices[w]

           y[i] = word_indices[next_word_list[index % len(sentence_list)]]

           index = index + 1

       yield x, y


# Functions from keras-team/keras/blob/master/examples/lstm_text_generation.py

def sample(preds, temperature=1.0):

   # helper function to sample an index from a probability array

   preds = np.asarray(preds).astype('float64')

   preds = np.log(preds) / temperature

   exp_preds = np.exp(preds)

   preds = exp_preds / np.sum(exp_preds)

   probas = np.random.multinomial(1, preds, 1)

   return np.argmax(probas)


def on_epoch_end(epoch, logs):

   # Function invoked at end of each epoch. Prints generated text.

   examples_file.write('\n----- Generating text after Epoch: %d\n' % epoch)


   # Randomly pick a seed sequence

   seed_index = np.random.randint(len(X_train+X_test))

   seed = (X_train+X_test)[seed_index]


   for diversity in [0.3, 0.4, 0.5, 0.6, 0.7]:

       sentence = seed

       examples_file.write('----- Diversity:' + str(diversity) + '\n')

       examples_file.write('----- Generating with seed:\n"' + ' '.join(sentence) + '"\n')

       examples_file.write(' '.join(sentence))


       for i in range(50):

           x_pred = np.zeros((1, MIN_SEQ))

           for t, word in enumerate(sentence):

               x_pred[0, t] = word_indices[word]


           preds = model.predict(x_pred, verbose=0)[0]

           next_index = sample(preds, diversity)

           next_word = indices_word[next_index]


           sentence = sentence[1:]

           sentence.append(next_word)


           examples_file.write(" "+next_word)

       examples_file.write('\n')

   examples_file.write('='*80 + '\n')

   examples_file.flush()

In [7]:
def get_model():

   print('Build model...')

   model = Sequential()

   model.add(Embedding(input_dim=len(words), output_dim=1024))

   model.add(Bidirectional(LSTM(128)))

   model.add(Dense(len(words)))

   model.add(Activation('softmax'))

   return model

In [8]:
def get_model():

   print('Build model...')

   model = Sequential()

   model.add(Embedding(input_dim=len(words), output_dim=1024))

   model.add(Bidirectional(LSTM(128)))

   model.add(Dense(len(words)))

   model.add(Activation('softmax'))

   return model

In [9]:
model = get_model()

model.compile(loss='sparse_categorical_crossentropy', optimizer="adam", metrics=['accuracy'])


file_path = "/content/drive/MyDrive/RapLyrics/Trap-Male/checkpoints/LSTM_LYRICS-epoch{epoch:03d}-words%d-sequence%d-minfreq%d-loss{loss:.4f}-acc{accuracy:.4f}-val_loss{val_loss:.4f}-val_acc{val_accuracy:.4f}" %(len(words), MIN_SEQ, MIN_FREQUENCY)


checkpoint = ModelCheckpoint(file_path, monitor='val_accuracy', save_best_only=True)

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

early_stopping = EarlyStopping(monitor='val_accuracy', patience=20)

callbacks_list = [checkpoint, print_callback, early_stopping]


examples_file = open('examples.txt', "w")

model.fit(generator(X_train, y_train, BATCH_SIZE),

                   steps_per_epoch=int(len(valid_seqs)/BATCH_SIZE) + 1,

                   epochs=1,

                   callbacks=callbacks_list,

                   validation_data=generator(X_test, y_train, BATCH_SIZE),

                   validation_steps=int(len(y_train)/BATCH_SIZE) + 1)

Build model...
9959/9959 [==============================] - 1400s 140ms/step - loss: 4.5980 - accuracy: 0.2195 - val_loss: 7.2258 - val_accuracy: 0.0422


INFO:tensorflow:Assets written to: /content/drive/MyDrive/RapLyrics/Trap-Male/checkpoints/LSTM_LYRICS-epoch001-words3385-sequence5-minfreq7-loss4.5980-acc0.2195-val_loss7.2258-val_acc0.0422/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/RapLyrics/Trap-Male/checkpoints/LSTM_LYRICS-epoch001-words3385-sequence5-minfreq7-loss4.5980-acc0.2195-val_loss7.2258-val_acc0.0422/assets


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=e1609fae-f7ed-42b7-8c74-772735f18851' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>